In [62]:
#
# Import Libraries
#

import pandas as pd
import os
import re
import numpy as np

In [65]:
root_path = r'S:\U_Proteomica\PROYECTOS\PESA_omicas'

mdata = pd.read_csv(
    r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\PESA_V2\WorkingFiles\main_metadata.tsv",
    sep='\t'
)

In [51]:
q2a = {}
for folder1 in os.listdir(root_path):
    if re.match('\da_Cohorte_\d+_V2', folder1):
        cohort = int(folder1[0])
        for folder2 in os.listdir(os.path.join(root_path, folder1,'Proteomics/TMT_Fraccionamiento')):
            if re.match('TMT\d+', folder2):
                #tmt = 'TMT'+re.search(r'TMT(\d+)', folder2).groups()[0]
                for folder3 in os.listdir(os.path.join(root_path, folder1,'Proteomics/TMT_Fraccionamiento', folder2, 'SanXoT')):
                    q2a[(cohort, f'{folder2}_{folder3}')] = pd.read_csv(
                        os.path.join(root_path, folder1,'Proteomics/TMT_Fraccionamiento', folder2, 'SanXoT', folder3, 'data/Q2A_outStats.xls'),
                        sep='\t'
                    )

In [84]:
xq = {}
for i in q2a:
    sn = mdata[np.logical_and(mdata.Cohort==i[0], mdata.Comparativa==i[1])].Seqn
    if sn.shape[0] == 1:
        xq[i] = pd.DataFrame()
        xq[i]['q'] = q2a[i]['idinf']
        xq[i]['x'] = q2a[i]['Xinf']-q2a[i]['Xsup']
        xq[i]['seqn'] = sn.values[0]

In [88]:
xq = pd.concat(xq.values())

In [91]:
xq = pd.pivot_table(xq, index='seqn', columns='q', values='x')

In [95]:
qdesc = xq.columns
fid = [re.search(r'\|([^|]+)\|', i).groups()[0] for i in xq.columns]

In [98]:
q2info = pd.DataFrame({'fid':fid, 'qdesc':qdesc})

In [99]:
xq.to_csv('Xq_minus_X.tsv', sep='\t', index=True)
q2info.to_csv('q2info.tsv', sep='\t', index=False)